In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

pd.set_option("display.max_colwidth", None)

In [3]:
#out = {}
out_support = {}

In [4]:
COL_info_detail = ["itemsets", "support", "tn", "fp", "fn", "tp"]

COL_info = ["itemsets", "support"]

# Parameters

In [45]:
K=3

metric = "d_error"
#t_value_col = "t_value_fp"

dataset_name = 'adult'

# Dataset

In [46]:
if dataset_name== "wine":


    
    from import_process_dataset import import_process_wine, train_classifier_kv

    df, class_map, continuous_attributes = import_process_wine()
    # # Train and predict with RF classifier

    df = train_classifier_kv(df)

elif dataset_name== "compas":
        from import_process_dataset import import_compas

        df, class_map, continuous_attributes = import_compas()

elif dataset_name== "adult":
    from import_process_dataset import import_process_adult, train_classifier_kv

    df, class_map, continuous_attributes = import_process_adult()

    df = train_classifier_kv(df, encoding = True)

In [47]:
min_sup_divergences = [0.01, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2]

## Divergence

# Tree divergence 

In [48]:
min_support_tree = 0.1

In [49]:
cols_c = ["class", "predicted"]

In [50]:
from divexplorer_generalized.FP_Divergence import FP_Divergence

## Tree divergence - divergence_criterion

In [51]:
type_criterion="divergence_criterion"

type_experiment = "one_at_time"

In [52]:
df_analyze = df.copy()

In [53]:
from tree_discretization import TreeDiscretization

tree_discr = TreeDiscretization()

# ## Extract tree
generalization_dict, discretizations = tree_discr.get_tree_discretization(
    df_analyze,
    type_splitting=type_experiment,
    min_support=min_support_tree,
    metric=metric,
    class_map=class_map,
    continuous_attributes=list(continuous_attributes),
    class_and_pred_names=cols_c,
    storeTree=True,
    type_criterion=type_criterion,
    # minimal_gain = 0.0015
)

In [54]:
tree_discr.printDiscretizationTrees(round_v =2, show_condition = True)

age
 root s=1.00 --> d_error=0.00
         age<=30 s=0.32 --> d_error=-0.10
                 age<=24 s=0.16 --> d_error=-0.14
                 age>24 s=0.15 --> d_error=-0.05
         age>30 s=0.68 --> d_error=0.04
                 age<=38 s=0.22 --> d_error=0.02
                         age<=34 s=0.11 --> d_error=0.02
                         age>34 s=0.11 --> d_error=0.03
                 age>38 s=0.46 --> d_error=0.06
                         age<=54 s=0.33 --> d_error=0.06
                                 age<=46 s=0.19 --> d_error=0.05
                                 age>46 s=0.14 --> d_error=0.08
                         age>54 s=0.13 --> d_error=0.03

hours-per-week
 root s=1.00 --> d_error=0.00
         hours-per-week<=40 s=0.70 --> d_error=-0.03
                 hours-per-week<=39 s=0.22 --> d_error=-0.09
                         hours-per-week<=29 s=0.11 --> d_error=-0.10
                         hours-per-week>29 s=0.11 --> d_error=-0.07
                 hours-per-week>39 s

# Divergence

In [55]:
true_class_name = 'class'
pred_class_name = 'predicted'

In [65]:
min_sup_divergences = [0.01, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2]

### Base

In [66]:
from utils_extract_divergence_generalized import (
    extract_divergence_generalized,
)

for min_sup_divergence in min_sup_divergences:
    FP_fm = extract_divergence_generalized(
        df_analyze,
        discretizations,
        generalization_dict,
        continuous_attributes,
        min_sup_divergence=min_sup_divergence,
        apply_generalization=False,
        true_class_name=true_class_name,
        predicted_class_name=pred_class_name,
        class_map=class_map,
        metrics_divergence = [metric],
        FPM_type="fpgrowth",
    )
   
    out_support.setdefault(min_sup_divergence, {})["base"] = max(FP_fm[metric])

### Generalized

In [67]:
from utils_extract_divergence_generalized import (
    extract_divergence_generalized,
)


for min_sup_divergence in min_sup_divergences:

    FP_fm = extract_divergence_generalized(
        df_analyze,
        discretizations,
        generalization_dict,
        continuous_attributes,
        min_sup_divergence=min_sup_divergence,
        apply_generalization=True,
        true_class_name=true_class_name,
        predicted_class_name=pred_class_name,
        class_map=class_map,
        metrics_divergence = [metric],
        FPM_type="fpgrowth",
    )
    
    fp_divergence_i = FP_Divergence(FP_fm, metric=metric)

    
    out_support.setdefault(min_sup_divergence, {})["generalized"] = max(FP_fm[metric])

1 10000
2 20000
1 10000


In [68]:
from utils_extract_divergence_generalized import (
    extract_divergence_generalized,
)


for min_sup_divergence in min_sup_divergences:

    FP_fm = extract_divergence_generalized(
        df_analyze,
        discretizations,
        generalization_dict,
        continuous_attributes,
        min_sup_divergence=min_sup_divergence,
        apply_generalization=True,
        true_class_name=true_class_name,
        predicted_class_name=pred_class_name,
        class_map=class_map,
        metrics_divergence = [metric],
        FPM_type="fpgrowth",
        take_top_k=1000
    )
    
    fp_divergence_i = FP_Divergence(FP_fm, metric=metric)
    
    out_support.setdefault(min_sup_divergence, {})["generalized_top"]  = max(FP_fm[metric])

1 10000
2 20000
1 10000


In [69]:
from utils_extract_divergence_generalized import (
    extract_divergence_generalized,
)

keep_items = tree_discr.get_keep_items_associated_with_divergence()

for min_sup_divergence in min_sup_divergences:

    FP_fm = extract_divergence_generalized(
        df_analyze,
        discretizations,
        generalization_dict,
        continuous_attributes,
        min_sup_divergence=min_sup_divergence,
        apply_generalization=True,
        true_class_name=true_class_name,
        predicted_class_name=pred_class_name,
        class_map=class_map,
        metrics_divergence = [metric],
        FPM_type="fpgrowth",
        keep_only_positive_divergent_items=keep_items,
        
    )
    
    fp_divergence_i = FP_Divergence(FP_fm, metric=metric)
    
    out_support.setdefault(min_sup_divergence, {})["generalized_pruned"]  = max(FP_fm[metric])

1 10000


In [70]:
from utils_extract_divergence_generalized import (
    extract_divergence_generalized,
)

keep_items = tree_discr.get_keep_items_associated_with_divergence()

for min_sup_divergence in min_sup_divergences:

    FP_fm = extract_divergence_generalized(
        df_analyze,
        discretizations,
        generalization_dict,
        continuous_attributes,
        min_sup_divergence=min_sup_divergence,
        apply_generalization=True,
        true_class_name=true_class_name,
        predicted_class_name=pred_class_name,
        class_map=class_map,
        metrics_divergence = [metric],
        FPM_type="fpgrowth",
        keep_only_positive_divergent_items=keep_items,
        take_top_k=1000
        
    )
    
    fp_divergence_i = FP_Divergence(FP_fm, metric=metric)
    
    out_support.setdefault(min_sup_divergence, {})["generalized_pruned_top"]  = max(FP_fm[metric])

1 10000


In [71]:
for s in out_support:
    print(s,  out_support[s]['generalized'] == out_support[s]['generalized_top'])
    print(s, 'p', out_support[s]['generalized_pruned'] == out_support[s]['generalized_pruned_top'])
    print(s,  out_support[s]['generalized'] == out_support[s]['generalized_pruned'])

0.1 True
0.1 p True
0.1 True
0.15 True
0.15 p True
0.15 True
0.2 True
0.2 p True
0.2 True
0.01 True
0.01 p True
0.01 True
0.02 True
0.02 p True
0.02 True
0.025 True
0.025 p True
0.025 True
0.03 True
0.03 p True
0.03 True
0.04 True
0.04 p True
0.04 True
0.05 True
0.05 p True
0.05 True
